# 1. Импорт библиотек.

In [1]:
# Все import соберем в начале проекта:

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import os
import seaborn as sns
import numpy as np

from skimpy import skim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Настроим вид таблиц:
pd.set_option("display.max_rows", 25)
pd.set_option("display.max_columns", 25)

# Чтобы полностью рассмотреть название продукта,
# снимем ограничения на количество символов в записи:
pd.set_option('display.max_colwidth', None)

In [2]:
# Загрузка файлов с данными.

df_dealer = pd.read_csv('data/marketing_dealer.csv', sep = ';')
df_dealerprice = pd.read_csv('data/marketing_dealerprice.csv', sep = ';')
df_product = pd.read_csv('data/marketing_product.csv', sep = ';')
df_productdealerkey = pd.read_csv('data/marketing_productdealerkey.csv', sep = ';')

In [3]:
display(df_dealer)

,id,name
0,1,Moi_vibor_WB
1,2,Akson
2,3,Bafus
3,5,Castorama
4,6,Cubatora
5,7,Komus
6,9,Megastroy
7,10,OnlineTrade
8,11,Petrovich
9,12,sdvor


In [4]:
skim(df_dealerprice)
df_dealerprice.sample(5)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 20416  │ │ string      │ 4     │                                                          │
│ │ Number of columns │ 7      │ │ int32       │ 2     │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓  │
│ ┃ column_name       ┃ NA   ┃ NA %    ┃ mean      ┃ sd      ┃ p0   ┃ p25     ┃ p75      ┃ p100     ┃ hist     ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩  │
│ │ id                │    0 │       0 │     10000 │    5900 │    2 │    5300 │    15000 │    21000 │  ██████  │  │
│ │ price             │    0 │       0 │      1300 │    2100 │    0 │     310 │     1200 │    34000 │    █     │  │
│ │ dealer_id         │    0 │       0 │       8.9 │     5.7 │    1 │       3 │       15 │       18 │  █▂▂▂▆▃  │  │
│ └───────────────────┴──────┴─────────┴───────────┴─────────┴──────┴─────────┴──────────┴──────────┴──────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                ┃ NA       ┃ NA %       ┃ words per row               ┃ total words             ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ product_key                │        0 │          0 │                           1 │                   21000 │  │
│ │ product_url                │      230 │        1.1 │                           1 │                   21000 │  │
│ │ product_name               │        0 │          0 │                           1 │                   21000 │  │
│ │ date                       │        0 │          0 │                           1 │                   21000 │  │
│ └────────────────────────────┴──────────┴────────────┴─────────────────────────────┴─────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

,id,product_key,price,product_url,product_name,date,dealer_id
17859,18010,1021731,817.0,https://www.sima-land.ru/1284631,"Универсальное моющее и чистящее средство Universal Spray, готовое к применению, 5 л",2023-07-28,13
7533,7595,90414456,564.0,https://spb.leroymerlin.ru/product/kraska-rezinovaya-prosept-073-1-cvet-korichnevyy-1-kg-90414456,Краска резиновая PROSEPT 073-1 цвет коричневый 1 кг,2023-07-17,8
14477,16834,874974462,1380.0,https://api.ozon.ru/composer-api.bx/page/json/v2?url=/seller/552340/products/?brand=140367694&layout_container=default&layout_page_index=4&miniapp=seller_552340&page=3&sold_out_page=1,Строительный антисептик Prosept,2023-07-26,17
14881,14934,23295728,2442.0,https://vimos.ru/product/antiseptik-dlya-naruzh-rabot-prosept-exterior-koncentrat-1-19-5l,Антисептик трудновымываемый Prosept Exterior концентрат 1:19 5л,2023-07-25,16
12510,12612,856324,992.0,https://akson.ru//p/ochistitel_epoksidnogo_naleta_prosept_epoxy_cleaner_gotovyy_sostav_0_5_l/,"Очиститель эпоксидного налета PROSEPT EPOXY CLEANER, готовый состав / 0,5 л",2023-07-24,2


In [5]:
skim(df_product)
df_product.head(5)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 496    │ │ string      │ 7     │                                                          │
│ │ Number of columns │ 15     │ │ float64     │ 6     │                                                          │
│ └───────────────────┴────────┘ │ int32       │ 2     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━┳━━━━━┳━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_na ┃     ┃      ┃           ┃            ┃           ┃            ┃           ┃            ┃        ┃  │
│ ┃ me        ┃ NA  ┃ NA % ┃ mean      ┃ sd         ┃ p0        ┃ p25        ┃ p75       ┃ p100       ┃ hist   ┃  │
│ ┡━━━━━━━━━━━╇━━━━━╇━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩  │
│ │ Unnamed:  │   0 │    0 │       250 │        140 │         0 │        120 │       370 │        500 │ ██████ │  │
│ │ 0         │     │      │           │            │           │            │           │            │        │  │
│ │ id        │   0 │    0 │       250 │        150 │         1 │        120 │       380 │        510 │ █████▇ │  │
│ │ ean_13    │  32 │  6.5 │ 470000000 │ 2700000000 │ 460000000 │ 4700000000 │ 470000000 │ 4700000000 │ ▂    █ │  │
│ │           │     │      │      0000 │          0 │      0000 │        000 │      0000 │        000 │        │  │
│ │ cost      │   5 │    1 │       880 │       1200 │        44 │        200 │       840 │       8000 │  █▁▁   │  │
│ │ recommend │   5 │    1 │      1900 │       2500 │         0 │        440 │      1800 │      15000 │  █▁▁   │  │
│ │ ed_price  │     │      │           │            │           │            │           │            │        │  │
│ │ category_ │  49 │  9.9 │        34 │         11 │        20 │         25 │        41 │         57 │ █▅▅▄▃▂ │  │
│ │ id        │     │      │           │            │           │            │           │            │        │  │
│ │ ozon_arti │ 130 │   26 │ 380000000 │  130000000 │ 190000000 │  230000000 │ 450000000 │  580000000 │ ▅  ▁█▁ │  │
│ │ cle       │     │      │           │            │           │            │           │            │        │  │
│ │ wb_articl │ 160 │   31 │ 150000000 │    2500000 │ 150000000 │  150000000 │ 150000000 │  170000000 │   █    │  │
│ │ e         │     │      │           │            │           │            │           │            │        │  │
│ └───────────┴─────┴──────┴───────────┴────────────┴───────────┴────────────┴───────────┴────────────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                 ┃ NA       ┃ NA %       ┃ words per row               ┃ total words            ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ article                     │        0 │          0 │                         1.1 │                    530 │  │
│ │ name                        │        2 │        0.4 

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,0,245,008-1,4.680010e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10 / 1 л,360.00,858.0,20.0,"Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.","Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.","Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.",189522705.0,150033482.0,008-1,NaN
1,1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN,NaN,NaN,NaN,NaN
2,2,443,0024-06 с,4.680010e+12,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,25.0,"Герметик акриловый для швов для деревянных домов, конструкций, изделий PROSEPT цвет сосна, ф/п 600мл","Герметик акриловый цвет сосна, ф/п 600мл","Герметик акриловый для швов для деревянных домов, конструкций, изделий PROSEPT цвет сосна, ф/п 600мл",189522735.0,150126217.0,0024-06-с,NaN
3,3,147,305-2,4.610090e+12,Кондиционер для белья с ароматом королевского ИрисаCrystal Rinserконцентрат / 2 л,157.73,342.0,29.0,"Кондиционер для белья ""Королевский Ирис"" Prosept Crystal Rinser, 2 л.","Кондиционер для белья ""Королевский Ирис"" Prosept Crystal Rinser, 2 л.","Кондиционер для белья ""Королевский Ирис"" Prosept Crystal Rinser, 2 л.",339377922.0,150032962.0,305-2,NaN
4,4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б,NaN


In [6]:
skim(df_productdealerkey)
df_productdealerkey.head(5)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 1700   │ │ int32       │ 3     │                                                          │
│ │ Number of columns │ 4      │ │ string      │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┓  │
│ ┃ column_name         ┃ NA    ┃ NA %     ┃ mean     ┃ sd     ┃ p0   ┃ p25    ┃ p75     ┃ p100    ┃ hist      ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━┩  │
│ │ id                  │     0 │        0 │     1100 │    620 │    1 │    430 │    1600 │    2000 │  █▄▆▆██   │  │
│ │ dealer_id           │     0 │        0 │       10 │    5.8 │    1 │      3 │      15 │      18 │  █▂▃▃▇▇   │  │
│ │ product_id          │     0 │        0 │      220 │    130 │    1 │    110 │     330 │     480 │  █▇▄█▆▅   │  │
│ └─────────────────────┴───────┴──────────┴──────────┴────────┴──────┴────────┴─────────┴─────────┴───────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ key                       │       0 │          0 │                            1 │                     1700 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106
2,3,546257,2,200
3,4,546408,2,38
4,5,651258,2,403


# Обработка имени товара.

In [7]:
# ДОБАВЛЕНО 27.11.2023


def clean_names_goods(my_series):
    '''
    Функция для очистки названий товаров выполняет следующие действия:
    - Перевод названии товаров в нижний регистр.
    - Удаление символа 'запятая' в именах товаров.
    - Удаление из имени товара повторяющихся пробелов.
    - Удаление из имени товара ненужных, одиноких символов.
    - Удаление пробелов в начале и в конце названия товара.
    '''   
    # Переведём все названия товаров в нижний регистр:
    my_series = my_series.str.lower()

    # Удалим символы 'запятая' в именах товаров:
    my_series = my_series.str.replace('[^\w\s]','')

    # Удалим символы 'слэш' в именах товаров:
    my_series = my_series.str.replace(' / ',' ')

    # Убираем из имени товара повторяющиеся пробелы:
    my_series = my_series.str.replace('\s+', ' ')

    # Убираем из имени товара ненужные символы:
    my_series = my_series.str.replace(' i ', ' ')
    my_series = my_series.str.replace(' l ', ' ')
    my_series = my_series.str.replace(' / ', ' ')
    my_series = my_series.str.replace('e ', ' ')

    # Исправим грамматическую ошибку:
    my_series = my_series.str.replace('univerasal', 'universal')

    # Приведём название Производителя в написании на английском:
    my_series = my_series.str.replace('просепт', 'prosept')

    # Уберём пробелы в начале и в конце информационных признаков:
    my_series = my_series.replace(r"^ +| +$", r"", regex=True)

    return my_series


def remove_names_goods(df, feature):
    '''
    Функция для удаления названий товаров выполняет следующие действия:
    - Удаление пропусков в названии товара.
    - Удаление дубликатов названий товаров.
    - Удалить недописанные названия товаров.
    '''
    values = [' ', '', 'оsb base', 'diona antibac']

    # Уберём пропуски в названии товара:
    df = df.dropna(subset=[feature])

    # Удалим дубликаты названий товаров:
    df = df.drop_duplicates(subset=[feature])

    # Удалить записи, в которых имена товаров - пробел, пусто ....
    df = df[df[feature].isin(values) == False ]

    return df


# Для решения задачи извлечения слов из сплошь записанного выражения
# не будем загружать огромные многомегабайтные библиотеки слов.
# В связи с ограниченным набором слов в названиях товаров, создадим свои,
# небольшие библиотеки, работающие практически мгновенно.


list_product_word = [
    'prosept', 'концентрат', 'crystal', 'готовый', 'duty', 'multipower', 
    'cooky', 'diona', 'готовое', 'ultra', 'antifoam', 'bath', 'universal', 
    'carpet', 'концентрированное', 'flox', 'эффектом', 'splash', 'epoxy', 
    'candy', 'optic', 'clean', 'шампунь', 'штуки'
    ]


def clean_name_products(row):
    '''
    Функция разделения предложения, написанных слитно, без пробелов.
    '''
    result = row['name']
    for word in list_product_word:
        tmp_str = result.split(str(word))
        if len(tmp_str) > 1:
            result = tmp_str[0] + ' ' + word + ' ' + tmp_str[1]

    return result


list_dealers_word = [
    'антижук', 'prosept', 'universal', 'ultra', 'grill', 'удаления',
    'floor', 'remover', 'средство', 'стекол', 'зеркал', 'пластика',
    'акриловых', 'bath', 'acryl', 'profi', 'eco', 'multipower', 'xm11',
    'graffiti', 'плесени', 'грибка', 'gel', 'снятия', 'shine', 'грунтовка',
    '20л', '10л', '2л', 'hand', 'cristal', 'против', 'лак', 'полуматовый',
    'глянцевый', 'невымываемый', 'машины', 'splash', 'орех', 'сlean', 'acid',
    'polish', 'удаления', 'hard', 'посуды', 'полов', 'комнат',  'spray',
    'посудомоечной', 'lime', 'rinser', 'sport', 'спортивной', 'черных',
    'black', 'сауны', 'бани', 'труб', 'засоров', 'extra', 'после',
    'очистки', 'ухода', 'мебелью', 'зеленый', 'красный', 'fungi'
    ]


def clean_dealers_name_products(row):
    '''
    Функция разделения предложения, написанных слитно, без пробелов.
    '''
    result = row['product_name']
    for word in list_dealers_word:
        tmp_str = result.split(word)
        if len(tmp_str) > 1:
            result = tmp_str[0] + ' ' + word + ' ' + tmp_str[1]

    return result


def preparation_dealer_data(df):
    '''
    Функция полной обработки и подготовки к проведению обучения
    названий товара дилеров.
    '''
    # Подготовим имена товаров ДИЛЕРОВ
    df['product_name'] = clean_names_goods(df['product_name'])
    df = remove_names_goods(df, 'product_name')
    # Разделим предложения, написанных слитно на слова
    df['product_name'] = df.apply(clean_dealers_name_products, axis=1)  
    # Почистим имена товаров дилеров:
    df['product_name'] = clean_names_goods(df['product_name'])
    return df


def preparation_product_data(df):
    '''
    Функция полной обработки и подготовки к проведению поиска
    названий товара производителя.
    '''
    # Проведём очистку названий товаров
    df['name'] = clean_names_goods(df['name'])
    # Удалим ошибочнве или пустые названия товара
    df = remove_names_goods(df, 'name')
    # Разделим предложения, написанных слитно на слова
    df['name'] = df.apply(clean_name_products, axis=1)
    # Проведём очистку названий товаров
    df['name'] = clean_names_goods(df['name'])
    return df


# Обработка Дилерских названий товаров.
df_dealerprice = preparation_dealer_data(df_dealerprice)

# Обработка названий товаров производителя.
df_product = preparation_product_data(df_product)

# Создание новых признаков.

In [8]:
# Добавим новый признак с длиной названия товара:

df_dealerprice['len_name'] = df_dealerprice['product_name'].str.len()
df_product['len_name'] = df_product['name'].str.len()

In [9]:
## Удаляем значения массы и объема из названия:

# Пример предподготовки списка запросов (обычно всего 1):

In [10]:
# Берём список запросов, а не один. С запасом!
question_str = ['антисептикнеВымываемыйпросепт l ultra, концентрат 110 / 1 л']
data = {'product_name': question_str}
question_df = pd.DataFrame(data) 
# display(question_df)

# Предподготовка, АНАЛОГИЧНАЯ train
question_df = preparation_dealer_data(question_df)

list_for_answer = question_df['product_name']
list_for_answer # ЭТОТ СПИСОК ПЕРЕДАВАТЬ НА ОБУЧЕНИЕ!

0    антисептик невымываемый prosept ultra концентрат 110 1 л
Name: product_name, dtype: object

In [11]:
list_for_answer[0]

'антисептик невымываемый prosept ultra концентрат 110 1 л'

# Векторизация. NearestNeighbors

In [12]:
tfidf_vectorizer = TfidfVectorizer()
tf_idf_train = (tfidf_vectorizer.fit_transform(
                df_dealerprice['product_name'].values.astype('U')))
tf_idf_targer = (tfidf_vectorizer.transform(
                 df_product['name'].values.astype('U')))

NUM_ANSWERS = 5 # Количество ответов на запрос == кол-во ближайших соседей.


# Александр, вноси список запросов 'list_for_answer' для поиска совпадений

# neigh.fit(tf_idf_train)
# distances, indices = neigh.kneighbors(tf_idf_targer)
# for i in range(0, 1):
#         print(f"Пример {i + 1} запроса ПРОИЗВОДИТЕЛЯ названия товара:",
#               f" '{df_product['name'][i]}'.")
        
#         neighbors = df_dealerprice['product_name'].iloc[indices[i]]
#         print("Варианты названий ДИЛЕРСКОГО:")        
#         print(neighbors,'\t','indices[i]:',indices[i],distances[i],"\n")

Пример 1 запроса ПРОИЗВОДИТЕЛЯ названия товара:  'антисептик невымываемый просепт ultra концентрат 110 1 л'.
Варианты названий ДИЛЕРСКОГО:
16839     антисептик невымываемый просепт ultra концентрат 110 1 л
18588     антисептик невымываемый просепт ultra концентрат 110 5 л
1092       антисептик невымываемый просепт ultra концентрат 110 5л
1072       антисептик невымываемый просепт ultra концентрат 110 1л
16856    антисептик невымываемый просепт ultra концентрат 110 20 л
Name: product_name, dtype: object 	 indices[i]: [1751 1890  829  809 1765] [0.         0.         0.42342242 0.45076649 0.51314085] 

Пример 2 запроса ПРОИЗВОДИТЕЛЯ названия товара:  'антигололед 32 просепт готовый состав 12 кг'.
Варианты названий ДИЛЕРСКОГО:
16909                                    грунт бетонконтакт просепт готовый состав 12 кг
1184                                      грунт просепт бетонконтакт готовый состав 6 кг
16947                                     грунт бетонконтакт просепт готовый состав 3 кг
31                         отбеливатель для древесины просепт eco 50 готовый состав 1 кг
1383     грунт бетоноконтакт просепт для гладких поверхностей готовый состав 12 кг 05110
Name: product_name, dtype: object 	 indices[i]: [1813  919 1849   31 1117] [0.78289557 0.99183003 0.99183003 1.03909655 1.04946797] 

Пример 3 запроса ПРОИЗВОДИТЕЛЯ названия товара:  'герметик акриловый цвет сосна фп 600мл'.
Варианты названий ДИЛЕРСКОГО:
13123            герметик акриловый межшовный для деревянных домов конструкций изделий просепт цвет белый фп 600мл
14480          герметик акриловый межшовный для деревянных домов конструкций изделий просепт цвет медовый фп 600мл
16726      герметик акриловый межшовный для деревянных домов конструкций изделий просепт цвет медовый фп 600мл 6шт
14482    герметик акриловый межшовный для деревянных домов конструкций изделий просепт цвет белый фп 600мл 12 штук
3734                                                         просепт герметик акриловый паропроницаемый 7 кг сосна
Name: product_name, dtype: object 	 indices[i]: [1671 1680 1711 1682 1430] [0.84896641 0.85754979 0.9187261  0.92703256 1.02682504]